# 28. 資料排除策略比較實驗

## 實驗目的

比較不同的資料排除策略對預測效能的影響。教授提問：初次已確診的個案是否應該排除？

## 實驗設計

| 組別 | 說明 | 樣本數 |
|------|------|--------|
| **A（目前做法）** | 包含所有樣本（含窗口起點已確診） | 13,514 |
| **B** | 排除首次健檢（Times=1）已確診的個案 | ~13,514（幾乎不變，Times=1 疾病率 ~0%） |
| **C** | 排除滑動窗口起點（Tinput1）已有目標疾病的樣本 | 依疾病而異 |

### 診斷閾值（Luo et al. 2024）

- 高血壓：SBP ≥ 140 mmHg 或 DBP ≥ 90 mmHg
- 高血糖：FBG ≥ 7.0 mmol/L
- 高血脂：TC ≥ 6.22 mmol/L

### 備註

由於原始資料集在 Times=1 時疾病率幾乎為 0%，方案 B 與方案 A 實質上無差異。  
因此本實驗主要比較 **A vs C**，方案 B 作為參考。

---

**對應章節**：Ch4 §4.X 資料篩選策略比較  
**Meeting 20 問題 1**：排除策略實驗（A/B/C 三組比較）

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print('套件載入完成')

套件載入完成


In [2]:
# === 載入資料 ===
df = pd.read_csv('../../data/01_primary/SUA/processed/SUA_sliding_window.csv')
print(f'資料集：{len(df):,} 筆樣本，{df["patient_id"].nunique():,} 名受檢者')
print(f'滑動窗口起點分佈：')
print(df['window_start'].value_counts().sort_index())

資料集：13,514 筆樣本，6,056 名受檢者
滑動窗口起點分佈：
1    6056
2    4302
3    2526
4     591
5      35
6       4
Name: window_start, dtype: int64


In [3]:
# === 定義特徵與目標 ===
base_features = ['sex', 'Age']
t1_features = ['FBG_Tinput1', 'TC_Tinput1', 'Cr_Tinput1', 'UA_Tinput1',
               'GFR_Tinput1', 'BMI_Tinput1', 'SBP_Tinput1', 'DBP_Tinput1']
t2_features = ['FBG_Tinput2', 'TC_Tinput2', 'Cr_Tinput2', 'UA_Tinput2',
               'GFR_Tinput2', 'BMI_Tinput2', 'SBP_Tinput2', 'DBP_Tinput2']
delta_features = ['Delta_FBG', 'Delta_TC', 'Delta_Cr', 'Delta_UA',
                  'Delta_GFR', 'Delta_BMI', 'Delta_SBP', 'Delta_DBP']

all_features = base_features + t1_features + t2_features + delta_features
print(f'特徵數：{len(all_features)}')

# 目標變數
target_map = {
    '高血壓': 'hypertension_target',
    '高血糖': 'hyperglycemia_target',
    '高血脂': 'dyslipidemia_target'
}

# Tinput1 疾病狀態（用於方案 C 排除）
disease_at_t1 = {
    '高血壓': (df['SBP_Tinput1'] >= 140) | (df['DBP_Tinput1'] >= 90),
    '高血糖': df['FBG_Tinput1'] >= 7.0,
    '高血脂': df['TC_Tinput1'] >= 6.22
}

# 首次健檢疾病狀態（用於方案 B 排除）
# Times=1 對應 window_start=1 且取 Tinput1 的值
# 但 Times=1 疾病率 ~0%，排除效果極微

print('\n=== Tinput1 已有疾病的樣本統計 ===')
for name, mask in disease_at_t1.items():
    print(f'{name}：{mask.sum():,} 筆 ({mask.mean()*100:.1f}%)')

特徵數：26

=== Tinput1 已有疾病的樣本統計 ===
高血壓：1,402 筆 (10.4%)
高血糖：367 筆 (2.7%)
高血脂：548 筆 (4.1%)


In [4]:
# === 建立三組資料集 ===

def get_datasets(df, target_name, target_col, disease_mask):
    """建立 A/B/C 三組資料集"""
    y_all = (df[target_col] == 2).astype(int)
    
    # 方案 A：全部樣本
    idx_a = df.index
    
    # 方案 B：排除首次健檢(window_start=1)中 Tinput1 已確診者
    # 即只排除「第一次進入研究時就已有疾病」的患者的所有窗口
    first_window = df[df['window_start'] == 1]
    patients_with_disease_at_first = first_window[disease_mask[first_window.index]]['patient_id'].unique()
    idx_b = df[~df['patient_id'].isin(patients_with_disease_at_first)].index
    
    # 方案 C：排除每個窗口中 Tinput1 已有目標疾病的樣本
    idx_c = df[~disease_mask].index
    
    datasets = {
        'A（全部樣本）': idx_a,
        'B（排除首次已確診）': idx_b,
        'C（排除窗口起點已確診）': idx_c
    }
    
    print(f'\n--- {target_name} ---')
    for name, idx in datasets.items():
        y_sub = y_all[idx]
        pos_rate = y_sub.mean() * 100
        print(f'{name}：{len(idx):,} 筆，陽性率 {pos_rate:.1f}%')
    
    return datasets, y_all

# 預覽三組資料集
for target_name, target_col in target_map.items():
    get_datasets(df, target_name, target_col, disease_at_t1[target_name])


--- 高血壓 ---
A（全部樣本）：13,514 筆，陽性率 19.3%
B（排除首次已確診）：13,341 筆，陽性率 19.0%
C（排除窗口起點已確診）：12,112 筆，陽性率 17.9%

--- 高血糖 ---
A（全部樣本）：13,514 筆，陽性率 5.9%
B（排除首次已確診）：13,514 筆，陽性率 5.9%
C（排除窗口起點已確診）：13,147 筆，陽性率 4.6%

--- 高血脂 ---
A（全部樣本）：13,514 筆，陽性率 7.9%
B（排除首次已確診）：13,504 筆，陽性率 7.9%
C（排除窗口起點已確診）：12,966 筆，陽性率 6.8%


In [5]:
# === 交叉驗證函式 ===

def run_cv(df_sub, features, y, groups, model_name='LR'):
    """執行 5-Fold CV 並回傳 AUC、Sensitivity、F1"""
    X = df_sub[features].values
    y_arr = y.values
    g_arr = groups.values
    
    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
    
    aucs, sens, f1s = [], [], []
    
    for train_idx, test_idx in cv.split(X, y_arr, g_arr):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y_arr[train_idx], y_arr[test_idx]
        
        if model_name == 'LR':
            model = LogisticRegression(
                max_iter=1000, class_weight='balanced', random_state=42
            )
        else:  # RF
            model = RandomForestClassifier(
                n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1
            )
        
        model.fit(X_train, y_train)
        y_prob = model.predict_proba(X_test)[:, 1]
        y_pred = model.predict(X_test)
        
        aucs.append(roc_auc_score(y_test, y_prob))
        sens.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
    
    return {
        'AUC': f'{np.mean(aucs):.3f} ± {np.std(aucs):.3f}',
        'AUC_mean': np.mean(aucs),
        'Sensitivity': f'{np.mean(sens):.3f}',
        'F1': f'{np.mean(f1s):.3f}'
    }

print('交叉驗證函式定義完成')

交叉驗證函式定義完成


In [6]:
# === 執行實驗：LR + RF × 3 疾病 × 3 組 ===

results = []

for target_name, target_col in target_map.items():
    datasets, y_all = get_datasets(df, target_name, target_col, disease_at_t1[target_name])
    
    for strategy_name, idx in datasets.items():
        df_sub = df.loc[idx].reset_index(drop=True)
        y_sub = y_all[idx].reset_index(drop=True)
        groups_sub = df.loc[idx, 'patient_id'].reset_index(drop=True)
        
        for model_name in ['LR', 'RF']:
            print(f'  {target_name} | {strategy_name} | {model_name}...')
            res = run_cv(df_sub, all_features, y_sub, groups_sub, model_name)
            results.append({
                '疾病': target_name,
                '排除策略': strategy_name,
                '模型': model_name,
                'AUC': res['AUC'],
                'AUC_mean': res['AUC_mean'],
                'Sensitivity': res['Sensitivity'],
                'F1': res['F1'],
                '樣本數': len(idx)
            })

results_df = pd.DataFrame(results)
print('\n實驗完成！')


--- 高血壓 ---
A（全部樣本）：13,514 筆，陽性率 19.3%
B（排除首次已確診）：13,341 筆，陽性率 19.0%
C（排除窗口起點已確診）：12,112 筆，陽性率 17.9%
  高血壓 | A（全部樣本） | LR...
  高血壓 | A（全部樣本） | RF...
  高血壓 | B（排除首次已確診） | LR...
  高血壓 | B（排除首次已確診） | RF...
  高血壓 | C（排除窗口起點已確診） | LR...
  高血壓 | C（排除窗口起點已確診） | RF...

--- 高血糖 ---
A（全部樣本）：13,514 筆，陽性率 5.9%
B（排除首次已確診）：13,514 筆，陽性率 5.9%
C（排除窗口起點已確診）：13,147 筆，陽性率 4.6%
  高血糖 | A（全部樣本） | LR...
  高血糖 | A（全部樣本） | RF...
  高血糖 | B（排除首次已確診） | LR...
  高血糖 | B（排除首次已確診） | RF...
  高血糖 | C（排除窗口起點已確診） | LR...
  高血糖 | C（排除窗口起點已確診） | RF...

--- 高血脂 ---
A（全部樣本）：13,514 筆，陽性率 7.9%
B（排除首次已確診）：13,504 筆，陽性率 7.9%
C（排除窗口起點已確診）：12,966 筆，陽性率 6.8%
  高血脂 | A（全部樣本） | LR...
  高血脂 | A（全部樣本） | RF...
  高血脂 | B（排除首次已確診） | LR...
  高血脂 | B（排除首次已確診） | RF...
  高血脂 | C（排除窗口起點已確診） | LR...
  高血脂 | C（排除窗口起點已確診） | RF...

實驗完成！


In [7]:
# === 結果總表 ===

print('=' * 80)
print('排除策略比較實驗結果')
print('=' * 80)

for model_name in ['LR', 'RF']:
    print(f'\n### {model_name} 模型')
    sub = results_df[results_df['模型'] == model_name]
    pivot = sub.pivot_table(
        index='排除策略', 
        columns='疾病', 
        values='AUC', 
        aggfunc='first'
    )[['高血壓', '高血糖', '高血脂']]
    print(pivot.to_string())
    print()

排除策略比較實驗結果

### LR 模型
疾病                      高血壓            高血糖            高血脂
排除策略                                                     
A（全部樣本）       0.712 ± 0.015  0.922 ± 0.017  0.858 ± 0.012
B（排除首次已確診）    0.716 ± 0.009  0.922 ± 0.017  0.860 ± 0.007
C（排除窗口起點已確診）  0.710 ± 0.011  0.910 ± 0.018  0.854 ± 0.011


### RF 模型
疾病                      高血壓            高血糖            高血脂
排除策略                                                     
A（全部樣本）       0.735 ± 0.012  0.924 ± 0.011  0.851 ± 0.017
B（排除首次已確診）    0.736 ± 0.009  0.924 ± 0.011  0.851 ± 0.007
C（排除窗口起點已確診）  0.748 ± 0.014  0.917 ± 0.014  0.844 ± 0.012



In [8]:
# === A vs C 差異分析 ===

print('=' * 60)
print('A vs C 差異分析（AUC）')
print('=' * 60)

for model_name in ['LR', 'RF']:
    print(f'\n### {model_name} 模型')
    for target_name in ['高血壓', '高血糖', '高血脂']:
        a_auc = results_df[
            (results_df['疾病'] == target_name) & 
            (results_df['排除策略'].str.startswith('A')) & 
            (results_df['模型'] == model_name)
        ]['AUC_mean'].values[0]
        
        c_auc = results_df[
            (results_df['疾病'] == target_name) & 
            (results_df['排除策略'].str.startswith('C')) & 
            (results_df['模型'] == model_name)
        ]['AUC_mean'].values[0]
        
        diff = (c_auc - a_auc) * 100
        direction = '↑' if diff > 0 else '↓' if diff < 0 else '→'
        print(f'  {target_name}：A={a_auc:.3f} → C={c_auc:.3f} ({direction}{abs(diff):.1f}%)')

A vs C 差異分析（AUC）

### LR 模型
  高血壓：A=0.712 → C=0.710 (↓0.3%)
  高血糖：A=0.922 → C=0.910 (↓1.1%)
  高血脂：A=0.858 → C=0.854 (↓0.4%)

### RF 模型
  高血壓：A=0.735 → C=0.748 (↑1.3%)
  高血糖：A=0.924 → C=0.917 (↓0.8%)
  高血脂：A=0.851 → C=0.844 (↓0.6%)


In [9]:
# === 陽性率變化（排除後的影響）===

print('=' * 60)
print('排除策略對陽性率的影響')
print('=' * 60)

for target_name, target_col in target_map.items():
    y_all = (df[target_col] == 2).astype(int)
    mask = disease_at_t1[target_name]
    
    # A
    rate_a = y_all.mean() * 100
    # C
    rate_c = y_all[~mask].mean() * 100
    
    print(f'{target_name}：A 陽性率 {rate_a:.1f}% → C 陽性率 {rate_c:.1f}%')

排除策略對陽性率的影響
高血壓：A 陽性率 19.3% → C 陽性率 17.9%
高血糖：A 陽性率 5.9% → C 陽性率 4.6%
高血脂：A 陽性率 7.9% → C 陽性率 6.8%


In [10]:
# === 儲存結果 ===
output_path = '../../results/exclusion_strategy_comparison.csv'
results_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f'結果已儲存至 {output_path}')

結果已儲存至 ../../results/exclusion_strategy_comparison.csv


## 實驗結論

### 方案 B vs A
- 首次健檢（Times=1）疾病率幾乎為 0%，排除首次已確診個案對結果無影響
- 原始資料集（Luo et al. 2024）已隱含「基線健康」的篩選條件

### 方案 C vs A
- 排除窗口起點已確診樣本後的 AUC 變化待觀察
- 若差異小（< 1%），佐證目前做法的合理性
- 若差異大，需在論文中討論並決定最終策略

### 論文整合
- 結果整合至 Ch4 消融實驗區（§4.X 資料篩選策略比較）
- 若差異小，也可作為 Ch5 研究限制的佐證